# Storage documents in Vector DB

API to load document in vector databases:



*   Chroma
*   Faiss
*   Weaviate
*   Pinecone



In [1]:
from datetime import datetime

In [ ]:
def load_document(name, src, type_document):
  return None

In [2]:
# document example to store vdb.
docs = [
    "This is a document about pineapple",
    "This is a document about oranges"
]
ids = ["id1", "id2"]

In [3]:
# environment to generate model embedding
from sentence_transformers import SentenceTransformer
model_embedding = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
query = "fruits with tropical flavor"

In [6]:
! pip install python-dotenv

In [12]:
import os
from dotenv import load_dotenv
load_dotenv(".env")

True

## Chroma

In [ ]:
#!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 89.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 7.9 MB/s eta 

In [ ]:
import chromadb

collection_name = "vbd_sample_collection"
chroma_client = chromadb.Client()
#chroma_client = chromadb.Client(Settings(persist_directory="./vector_db")) #Storage in local directory.
collection = chroma_client.get_or_create_collection(
    name= collection_name,
     metadata={
        "description": "my first Chroma collection",
        "created": str(datetime.now())
    }  )

collection.upsert(documents=docs,ids=ids)

In [ ]:
results = collection.query(
    query_texts=[query], # Chroma will embed this for you
    n_results=2 # how many results to return
)

print(results)

collections = chroma_client.list_collections()

print(collections)

{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[0.1796746551990509, 1.1437273025512695]]}
[Collection(name=vbd_collection), Collection(name=vbd_sample_collection), Collection(name=vdb_collection)]


## Faiss

In [ ]:
import faiss
import numpy as np
import pickle



In [ ]:
src_index="faiss_index.idx"
src_texts="faiss_docs.pkl"
src_ids="faiss_ids.pkl"

In [ ]:
embeddings = model_embedding.encode(docs)

dim = embeddings.shape[1]

index = faiss.IndexFlatL2(dim)

index.add(np.array(embeddings))

faiss.write_index( index, src_index)

In [ ]:
with open(src_texts, "wb") as f:
      pickle.dump(docs, f)

with open(src_ids, "wb") as f:
    pickle.dump(ids, f)

print(f"Saved: {src_index}, {src_texts}, {src_ids}")

Guardado: faiss_index.idx, faiss_docs.pkl, faiss_ids.pkl


In [ ]:
# Loads ids and docs
index = faiss.read_index("faiss_index.idx")
with open("faiss_docs.pkl", "rb") as f:
    docs = pickle.load(f)
with open("faiss_ids.pkl", "rb") as f:
    ids = pickle.load(f)

# Search
query_emb = model_embedding.encode([query]).astype("float32")
D, I = index.search(query_emb, k=2)

# Show resuls
for idx in I[0]:
    print(f"ID: {ids[idx]} - Texto: {docs[idx]}")


ID: id1 - Texto: This is a document about pineapple
ID: id2 - Texto: This is a document about oranges


## Weaviate

In [ ]:
! pip install weaviate-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.0/433.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-ai-generativelanguage 0.6.15 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.2, but you have protobuf 6.31.1 which is incompatible.
ydf 0.12.0 requires protobuf<6.0.0,>=5.29.1, but you have protobuf 6.31.1 which is inc

In [ ]:
import weaviate
from weaviate.classes.init import Auth
import uuid

In [ ]:
# Best practice: store your credentials in environment variables
weaviate_url = os.getenv("WEAVIATE_URL")
weaviate_key = os.getenv("WEAVIATE_API_KEY")

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,                     # Weaviate URL: "REST Endpoint" in Weaviate Cloud console
    auth_credentials=Auth.api_key(weaviate_key),  # Weaviate API key: "ADMIN" API key in Weaviate Cloud console
)

print(client.is_ready())  # Should print: `True`

#client.close()

True


In [ ]:
embeddings = model_embedding.encode(docs).tolist()

In [ ]:
from weaviate.classes.config import Configure

name_collection = "DemoCollection"

client.collections.create(
    name=name_collection,
    vectorizer_config=None
    # Additional parameters not shown
)

collection = client.collections.get(name_collection)

print(collection)


<weaviate.Collection config={
  "name": "DemoCollection",
  "description": null,
  "generative_config": null,
  "inverted_index_config": {
    "bm25": {
      "b": 0.75,
      "k1": 1.2
    },
    "cleanup_interval_seconds": 60,
    "index_null_state": false,
    "index_property_length": false,
    "index_timestamps": false,
    "stopwords": {
      "preset": "en",
      "additions": null,
      "removals": null
    }
  },
  "multi_tenancy_config": {
    "enabled": false,
    "auto_tenant_creation": false,
    "auto_tenant_activation": false
  },
  "properties": [],
  "references": [],
  "replication_config": {
    "factor": 1,
    "async_enabled": false,
    "deletion_strategy": "NoAutomatedResolution"
  },
  "reranker_config": null,
  "sharding_config": {
    "virtual_per_physical": 128,
    "desired_count": 1,
    "actual_count": 1,
    "desired_virtual_count": 128,
    "actual_virtual_count": 128,
    "key": "_id",
    "strategy": "hash",
    "function": "murmur3"
  },
  "vector_in

In [ ]:
# Save docuemtns
for i in range(len(docs)):
    collection.data.insert(
        properties={"text": docs[i]},
        vector=embeddings[i],
        uuid=str(uuid.uuid4())
    )

print("documents registered")


Documentos insertados manualmente con vectores.


In [ ]:
from weaviate.classes.query import MetadataQuery

vector = model_embedding.encode([query])[0].tolist()

#search

result = collection.query.near_vector(
    near_vector=vector, # your query vector goes here
    limit=3,
    return_metadata=MetadataQuery(distance=True)
    )

# Show results
for o in result.objects:
    print(o.properties)
    print(o.metadata.distance)


In [ ]:
client.close()

## Pinecone

In [5]:
! pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 18.0 MB/s eta 0:00:00


In [13]:
# Import the Pinecone library
from pinecone import Pinecone

pinecone_key = os.getenv("PINECONE_API_KEY")

# Initialize a Pinecone client with your API key
pc = Pinecone(api_key=pinecone_key)


In [16]:
from pinecone import ServerlessSpec

# Create a dense index with integrated embedding
index_name = "vbd-sample-collection"

embeddings = model_embedding.encode(docs).tolist()

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=len(embeddings[0]),
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )

In [17]:
# Get idx
index = pc.Index(index_name)

# Insert docs
items = [(f"id{i}", embeddings[i], {"text": docs[i]}) for i in range(len(docs))]
index.upsert(vectors=items)

{'upserted_count': 2}

In [19]:
# search query
query_vector = model_embedding.encode([query])[0].tolist()

# similiarity search
results = index.query(
    vector=query_vector,
    top_k=3,
    include_metadata=True
)

for match in results["matches"]:
    score = match["score"]
    text = match["metadata"]["text"]
    print(f"Tex: {text}\n Score: {score:.3f}\n---")



Tex: This is a document about pineapple
 Score: 0.461
---
Tex: This is a document about oranges
 Score: 0.409
---
